# Import

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from datetime import datetime
from scipy.signal import savgol_filter

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, BatchNormalization, LSTM, RepeatVector, ConvLSTM1D, Flatten
from keras.models import Model
from keras.models import model_from_json
from keras import regularizers
import keras
import tensorflow as tf

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=1e-3, patience=8, verbose=1, mode='auto',
    baseline=None, restore_best_weights=True)

from modules.indicators import compute_lagging_regression

# Get data

In [ ]:
symbol= 'BTC-USDT'

df_BTC = download_historical_data(symbol,'1hour').iloc[-3000:]
df_BTC["Return"] = df_BTC.Close.pct_change()
df_BTC.dropna(inplace=True)

df_BTC['Close_filtered'] = savgol_filter(df_BTC['Close'].values, 25, 2,mode='nearest')
print(df_BTC.shape)
df_BTC.head()

# Trying to denoise.

## Normalize data

In [ ]:
WINDOW = 50
INDEXING_WINDOW = 1

df_BTC['Close_filtered_norm'] = (df_BTC.Close_filtered-df_BTC.Close_filtered.mean())/df_BTC.Close_filtered.std()
df_BTC['Close_norm'] = (df_BTC.Close-df_BTC.Close.mean())/df_BTC.Close.std()

## Train and Test training

In [ ]:
#df_BTC =  df_BTC.iloc[-5000:]

X = np.array([df_BTC.Close_norm.iloc[i:i+WINDOW].values.reshape(1,-1) for i in range(len(df_BTC)-WINDOW)])[:-INDEXING_WINDOW]
y = np.array([df_BTC.Close_filtered_norm.iloc[i:i+WINDOW].values.reshape(1,-1) for i in range(len(df_BTC)-WINDOW)])[INDEXING_WINDOW:]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, shuffle=True)

In [ ]:
fig, ax_left = plt.subplots(1, figsize=(25,8))

ax_left.plot(list(range(len(X[0][0]))),X[0][0]*df_BTC.Close_filtered.std()+df_BTC.Close_filtered.mean(),color='green',label='X1')
ax_left.plot([i+INDEXING_WINDOW for i in range(len(X[INDEXING_WINDOW][0]))],X[INDEXING_WINDOW][0]*df_BTC.Close_filtered.std()+df_BTC.Close_filtered.mean(),color='green',label='X2', linestyle='--')
ax_left.plot([i+INDEXING_WINDOW for i in range(len(y[INDEXING_WINDOW][0]))],y[0][0]*df_BTC.Close_filtered.std()+df_BTC.Close_filtered.mean(),color='blue',label='y')
ax_left.legend(loc="upper left")
print(X_train.shape)

### Only dense

In [ ]:
input_window = Input(shape=(1,WINDOW))
encoded = Dense(10, activation='relu')(input_window)
inter_1 = Dense(30, activation='relu')(encoded)
decoded = Dense(WINDOW, activation='linear')(inter_1)
autoencoder = Model(input_window, decoded)
encoder = Model(input_window, encoded)


autoencoder.summary()
autoencoder.compile(optimizer='adam', loss='mse')

history = autoencoder.fit(X, y,
                epochs=200,
                batch_size=1024,
                shuffle=True,
                validation_split=0.2,
                callbacks=[early_stop])

### With LSTM

In [ ]:
input_window = Input(shape=(1,WINDOW))
encoded = LSTM(4, kernel_initializer='he_uniform', return_sequences=True, name='encoder_1')(input_window)
encoded_2 = Dense(10, activation='relu',)(encoded)
norm_1 = BatchNormalization()(encoded_2)
inter_1 = Dense(30, activation='relu')(norm_1)
norm_2 = BatchNormalization()(inter_1)
decoded = Dense(WINDOW, activation='linear')(norm_2)
autoencoder = Model(input_window, decoded)
encoder = Model(input_window, encoded)


autoencoder.summary()
autoencoder.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

history = autoencoder.fit(X, y,
                epochs=200,
                batch_size=1024,
                shuffle=True,
                validation_split=0.2,
                callbacks=[early_stop])

In [ ]:
predictions = autoencoder.predict(X)
print(len(X))
print(len(predictions))

In [ ]:
from random import randint
index = -randint(0,len(y)-1)

fig, ax_left = plt.subplots(1, figsize=(25,8))


ax_left.plot(list(range(len(y[index][0]))),y[index][0]*df_BTC.Close_filtered.std()+df_BTC.Close_filtered.mean(),color='green',label='Y')
ax_left.plot(list(range(len(predictions[index][0]))),savgol_filter(predictions[index][0]*df_BTC.Close_filtered.std()+df_BTC.Close_filtered.mean(), 25, 2,mode='nearest'),color='blue',label='Y pred')
ax_left.legend(loc="upper left")